In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.tabular.all import *

In [ ]:
train_df = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')
test_df = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')

In [ ]:
train_df.drop(['ID_code'], axis=1, inplace=True)

In [ ]:
test_df_ids = test_df.pop('ID_code')

In [ ]:
test_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
len_train_df = len(train_df)
cont_names = list(train_df.columns[1:])
procs = [FillMissing, Normalize]
splits = RandomSplitter(valid_pct=0.01)(range_of(train_df))

In [ ]:
to = TabularPandas(train_df, procs=[FillMissing, Normalize],
                   cont_names = cont_names,
                   y_names='target',
                   splits=splits)

In [ ]:
dls = to.dataloaders(bs=64)

In [ ]:
test_dl = dls.test_dl(test_df)

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, 2e-3, cbs=SaveModelCallback(monitor='accuracy'))

In [ ]:
preds = learn.get_preds(dl=test_dl)

In [ ]:
preds = to_np(preds[0])

In [ ]:
test_df['target'] = preds

In [ ]:
test_df['ID_code'] = test_df_ids

In [ ]:
test_df.to_csv('submission2.csv', columns=['ID_code', 'target'], index=False)